In [11]:
import QUANTAXIS as QA

In [12]:
import QAStrategy
from QAStrategy.qactabase import QAStrategyCTABase

In [13]:
class x(QAStrategyCTABase):
    def on_bar(self, data):
        
        print(data)

Create new Account

In [14]:
account = x(code= 'rb2005',frequence='1min', strategy_id='testfordebug4')

In [8]:
account.send_order('BUY','OPEN',3350, 1)

AttributeError: 'x' object has no attribute 'running_mode'

In [10]:
dir(QAStrategyCTABase)

['BarsSinceEntryLong',
 'BarsSinceEntryShort',
 'EntryPriceLong',
 'EntryPriceShort',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_debug_sim',
 '_on_1min_bar',
 'add_subscriber',
 'bar_id',
 'callback',
 'check_order',
 'control_status',
 'daily_func',
 'debug',
 'debug_currenttick',
 'debug_histick',
 'debug_sim',
 'debug_t0',
 'force_close',
 'get_cash',
 'get_code',
 'get_code_marketdata',
 'get_current_marketdata',
 'get_exchange',
 'get_positions',
 'ind2str',
 'load_strategy',
 'market_data',
 'market_datetime',
 'old_data',
 'on_15min_bar',
 'on_1min_bar',
 'on_30min_bar',
 'on_5min_bar',
 'on_bar',
 'on_dailyclose',
 'on_dailyopen',
 'on_deal',
 'on_ordere

In [17]:
from QAStrategy.qastockbase import QAStrategyStockBase

In [19]:
#example
import random
class strategy(QAStrategyStockBase):
    def on_bar(self, data):
        print(data)

        
        code = data.name[1]
        pos = self.get_positions(code)
        print('---------------under is 当前全市场的market_data --------------')
        
        print(self.get_current_marketdata())
        print('---------------under is 当前品种的market_data --------------')
        print(self.get_code_marketdata(code))
        print(code)
        
        
        
        if random.random() > 0.5:
            self.send_order('BUY', 'OPEN', code, price=data['close'], volume=1000)
        else:
            if pos.volume_long > 0:
                self.send_order('SELL', 'CLOSE', code, price=data['close'], volume=pos.volume_long)
            
        print('---------------under is 账户情况和持仓情况 -------------------')
        print(self.get_positions(code).message)
        print(self.acc.history_table)
            
        #print(self.running_time)
        input()

In [ ]:
#example 
from QAStrategy import QAStrategyCTABase
import QUANTAXIS as QA

class CCI(QAStrategyCTABase):
    def on_bar(self, bar):
        """你的大部分策略逻辑都是在此写的
        """
        res = self.cci()  
        print(res.iloc[-1])
        if res.CCI[-1] < -100:
            print('LONG')
            if self.positions.volume_long == 0:
                self.send_order('BUY', 'OPEN', price=bar['close'], volume=1)
            if self.positions.volume_short > 0:
                self.send_order('BUY', 'CLOSE', price=bar['close'], volume=1)

        elif res.CCI[-1] > 100:
            print('SHORT')
            if self.positions.volume_short == 0:
                self.send_order('SELL', 'OPEN', price=bar['close'], volume=1)
            if self.positions.volume_long > 0:
                self.send_order('SELL', 'CLOSE', price=bar['close'], volume=1)

    def cci(self,):
        """你可以自定义你想要的函数
        """
        return QA.QA_indicator_CCI(self.market_data, 61)


strategy = CCI(code='rb2005', frequence='1min',
                strategy_id='a3916de0-bd28-4b9c-bea1-94d91f1744ac')
strategy.run_backtest()